In [16]:
import pandas as pd
from pandas.io.html import read_html
import numpy as np
import os
from google.cloud import bigquery
from google.cloud import storage 
import re
from google.cloud.bigquery import SchemaField

def get_biden_onion(): 
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/zachquinn/Downloads/ornate-reef-332816-a7425b762ba2.json"
    
    joe = 'https://en.wikipedia.org/wiki/Joe_Biden_(The_Onion)'
    joe_wiki = pd.read_html(joe, attrs={'class': 'wikitable'})
    biden_onion = joe_wiki[0]
    biden_onion = biden_onion.drop(labels=(76), axis=0)
    for column in biden_onion.columns:
        biden_onion[column] = biden_onion[column].str.replace(r'\[.*\]', ' ')
        biden_onion['POTUS'] = biden_onion['POTUS'].str.replace(r'(first term)', '')
        biden_onion['POTUS'] = biden_onion['POTUS'].str.replace(r"\(.*\)", '')
        biden_onion['POTUS'] = biden_onion['POTUS'].str.replace(r'Trump transition', '')
    biden_onion = biden_onion.drop('#', axis=1)
    biden_onion['Date'] = pd.to_datetime(biden_onion['Date'])
    biden_onion['POTUS'] = biden_onion['POTUS'].astype('category')
    biden_onion['Type'] = biden_onion['Type'].astype('category')
    biden_onion['Year'] = biden_onion['Year'].astype(int)
    biden_onion['Headline_Length'] = biden_onion['Headline'].str.len()
    biden_onion = pd.DataFrame(biden_onion)
    
    client = bigquery.Client()
    dataset_id = 'biden_onion'
    table_id = 'headlines'
    
    dataset_ref = client.dataset(dataset_id)
    table_id = dataset_ref.table(table_id)
    
    schema = [
        bigquery.SchemaField("POTUS", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Year", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("Date", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("Headline", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("Type", "STRING", mode="NULLABLE")
    ]
    
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.schema = schema
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=True
    job_config.ignore_unknown_values=True 

    job = client.load_table_from_dataframe(
    biden_onion,
    table_id,
    location='US',
    job_config=job_config)
    
    job.result()
    
    return print('Biden Onion Headlines Table Has Loaded.')
    
get_biden_onion()

Biden Onion Headlines Table Has Loaded.
